# Part III: Code Project
## Question 1: Merkle Tree Implementation

Our program will prompt the user to enter a series of text files with their paths and read the contents of those files into a list.  It creates a "Node" class to store the data values from the list, and  "merkleTree", which contains functions to construct a merkle tree with those nodes.  The tree's leaf nodes are populated with the hashes of the text data from the user-provided files. The parent nodes of each two of the leaf nodes are the hashes of those two values, and this recurses to a single value. The MerkleTree class also contains a function to find just the root hash value for its built-in verification function for comparing two lists. The display function for this class, "printTree", will print out the node values in their corresponding levels of the tree.  For testing, there is also a utility function to change the texts in one of the provided files for comparison with the original files.     



### 4 Text Files

In [5]:
# modified from merkle tree basic implementation from https://github.com/andipro123/merkle-tree-Python/blob/master/merkleTree.py

import hashlib

#function to read in and store text file paths and  names
def collect_files():
    file_list = []
    done = False
    while done == False:
        file_name = input("Please enter file name and path or type 'done': ")
        if file_name == 'done':
            done = True
        else:
            file_list.append(file_name)
    print("Done with list")
    return(file_list)

#function to read in list of file names and  paths  from collect files
#creating separate function so user does not have to type in list of file names twice to compare
def readinfiles(listofnames):
    file_list = []
    for i in range(len(listofnames)):
        with open(listofnames[i], 'r') as file:
            file_object = file.read()
            file_list.append(file_object)
    return(file_list)
        
#function to alter text in one of the input files
def change_text():
    file_name = input("Please enter file name and path to change: ")
    with open (file_name, 'w') as file:
        new_text = input("Please enter the new text: ")
        file.write(new_text)
    return()

#test flat input list 
quicktestlist = ['test1', 'test2', 'test3', 'test4', 'test5', 'test6']
#for testing verifyUtil function
quickverifytestlist = ['test1', 'test2', 'test3', 'test4', 'test5', 'test7']

# Node class that represents a node of a binary tree
class Node:
    #function to initialise the node with data, left child and right child, representing a binary
    #tree
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
    
    #utility function to check if a given node has 2 children
    def isFull(self):
        return self.left and self.right
    
    #utility function to print the contents of the node
    def __str__(self):
        return self.data
    
    #utility function to check if a given node has 0 children (is a leaf node)
    def isLeaf(self):
        return ((self.left == None) and (self.right == None))

# Merkle tree class for actual implementation of the tree
class merkleTree:

    # function to initialise the tree with "None" as root and merkleRoot set to blank
    def __init__(self, arr):
        self.root = None
        self._merkleRoot = ''
        self.makeTreeFromArray(arr)
        self.calculateMerkleRoot()
        
    # private utility function to return a hash of an input string, using the sha256 algorithm    
    def __returnHash(self, x):
            return (hashlib.sha256(x.encode()).hexdigest())
        
    # function to generate a tree from a list. The list is expected to contain
    # the list of transactions(as strings) we wish to seal in a block    
    def makeTreeFromArray(self, arr):
        arr = arr.copy()
        # all the elements of the list must be leaf nodes, hence total nodes in the tree
        # is [2*(noOfLeafNodes) - 1]
        def __noOfNodesReqd(arr):
            x = len(arr)
            return (2*x - 1)
        
        # Private function to build tree with a list generated by a sequence corresponding to the
        # number of nodes required for the tree.
        # For e.g. if we have 4 transactions, i.e. 4 leafNodes, we will have a total of 7 nodes
        # in the tree. Hence this function takes in a list that looks like [1,2,3,4,5,6,7] and 
        # creates a binary tree by recursively inserting elements in inorder fashion.
        def __buildTree(arr, root, i, n): 
      
            # Base case for recursion  
            if i < n: 
                temp = Node(str(arr[i]))  
                root = temp  

                # insert left child  
                root.left = __buildTree(arr, root.left,2 * i + 1, n)  

                # insert right child  
                root.right = __buildTree(arr, root.right,2 * i + 2, n) 
                
            return root
        
        # This function adds our transactions (leaf Node data) to the Leaf Nodes in the tree
        # hence now the tree looks like :
                #                 ""
                #                 /\
                #         ""              ""
                #         /\              /\
                #    (txn1) (txn2)  (txn3) (txn4)      
        def __addLeafData(arr, node):
            
            if not node:
                return
            
            __addLeafData(arr, node.left)
            if node.isLeaf():
                node.data = self.__returnHash(arr.pop(0))
            else:
                node.data = ''
            __addLeafData(arr, node.right)
        
        #Driver function calls of the main function makeTreeFromArray
        nodesReqd = __noOfNodesReqd(arr)
        nodeArr = [num for num in range(1,nodesReqd+1)]
        self.root = __buildTree(nodeArr, None, 0, nodesReqd)
        __addLeafData(arr,self.root)

    # utility function to traverse the tree using inorder Traversal algorithm, must provide <the_name_of_the_tree_object>.root     
    def inorderTraversal(self, node):
        if not node:
            return []
        
        left = self.inorderTraversal(node.left)
        right = self.inorderTraversal(node.right)
        return left + [node.data] + right
    
    # function to calculate merkle root of the tree. This is a recursive algorithm.
    # Essentially, the data of parent node P, is the hash of concatenation of data of its two 
    # child nodes, A and B. If H(x) is the hash function,
    # P = H( H(A) + H(B) )
    # If R is the root, then data of R is the merkle hash or merkle root.  
    def calculateMerkleRoot(self):
         
        def __merkleHash(node):
            if node.isLeaf():
                return node
            
            left = __merkleHash(node.left).data
            right = __merkleHash(node.right).data
            node.data = self.__returnHash(left+right)
            return node
        
        merkleRoot = __merkleHash(self.root)
        self._merkleRoot = merkleRoot.data
        
        return self._merkleRoot 

    #utility function to get the merkle Root of that tree
    def getMerkleRoot(self):
        return self._merkleRoot
    
    # utility function to verify if the transactions are intact with respect to this tree.
    # Takes in a new list, creates a new merkle tree, calculate its merkle root,
    # and returns if merkle roots match. If they match, transaction list is intact and verified.
    # If transactions are tampered or changed, this function returns False.
    def verifyUtil(self, arr1):
        hash1 = self.getMerkleRoot()
        new_tree = merkleTree(arr1)
        hash2 = new_tree.getMerkleRoot()
        if hash1 == hash2:
            print(f"Hash 1: {hash1[:12]}, Hash 2: {hash2[:12]}") #hash shortened for readability
            print("Texts verified successfully")
            return True
        else:
            print(f"Hash 1: {hash1[:12]}, Hash 2: {hash2[:12]}") #hash shortened for readability
            print("Texts have been changed")
            return False
    
    def printTree(self):
        from collections import deque

        if not self.root:
            print("Empty tree.")
            return

        q = deque()
        q.append(self.root)
        level = 0
        while q:
            level_size = len(q)
            print(f"Level {level}: ", end='')
            for _ in range(level_size):
                node = q.popleft()
                print(f"{node.data[:10]} ", end='')  # Shorten the hash for readability
                if node.left:
                    q.append(node.left)
                if node.right:
                    q.append(node.right)
            print()
            level += 1


# make two lists from reading in files, change text in one of the files,
# create second list with same files but one with altered content
file_names = collect_files()
file_list1 = readinfiles(file_names)
change_text()
file_list2 = readinfiles(file_names)

#create the merkletrees
testtree1 = merkleTree(file_list1)
testtree2 = merkleTree(file_list2)

#Print merkle trees for original set of files and altered files
print("\n" +"Merkle tree of original files:")
testtree1.printTree()
print("\n" + "Merkle tree of same files with one altered text:")
testtree2.printTree()

#comparing the root hashes
print("\n" + "Comparing root hashes:")
#compare file_list1 against itself 
test1 = testtree1.verifyUtil(file_list1)
#compare file_list1 against file_list2
test2 = testtree1.verifyUtil(file_list2)


Please enter file name and path or type 'done':  project4part3question1testinput1.txt
Please enter file name and path or type 'done':  project4part3question1testinput2.txt
Please enter file name and path or type 'done':  project4part3question1testinput3.txt
Please enter file name and path or type 'done':  project4part3question1testinput4.txt
Please enter file name and path or type 'done':  done


Done with list


Please enter file name and path to change:  project4part3question1testinput4.txt
Please enter the new text:  New random text!



Merkle tree of original files:
Level 0: e0c3a06a0c 
Level 1: 2f297f1520 2ae11d0915 
Level 2: 1b4f0e9851 60303ae22b fd61a03af4 8845a27d83 

Merkle tree of same files with one altered text:
Level 0: 323c6f387d 
Level 1: 2f297f1520 a2233f9fdf 
Level 2: 1b4f0e9851 60303ae22b fd61a03af4 9c64347be9 

Comparing root hashes:
Hash 1: e0c3a06a0cb5, Hash 2: e0c3a06a0cb5
Texts verified successfully
Hash 1: e0c3a06a0cb5, Hash 2: 323c6f387d1c
Texts have been changed


### 6 Test Files

In [6]:
# modified from merkle tree basic implementation from https://github.com/andipro123/merkle-tree-Python/blob/master/merkleTree.py

import hashlib

#function to read in and store text file paths and  names
def collect_files():
    file_list = []
    done = False
    while done == False:
        file_name = input("Please enter file name and path or type 'done': ")
        if file_name == 'done':
            done = True
        else:
            file_list.append(file_name)
    print("Done with list")
    return(file_list)

#function to read in list of file names and  paths  from collect files
#creating separate function so user does not have to type in list of file names twice to compare
def readinfiles(listofnames):
    file_list = []
    for i in range(len(listofnames)):
        with open(listofnames[i], 'r') as file:
            file_object = file.read()
            file_list.append(file_object)
    return(file_list)
        
#function to alter text in one of the input files
def change_text():
    file_name = input("Please enter file name and path to change: ")
    with open (file_name, 'w') as file:
        new_text = input("Please enter the new text: ")
        file.write(new_text)
    return()

#test flat input list 
quicktestlist = ['test1', 'test2', 'test3', 'test4', 'test5', 'test6']
#for testing verifyUtil function
quickverifytestlist = ['test1', 'test2', 'test3', 'test4', 'test5', 'test7']

# Node class that represents a node of a binary tree
class Node:
    #function to initialise the node with data, left child and right child, representing a binary
    #tree
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
    
    #utility function to check if a given node has 2 children
    def isFull(self):
        return self.left and self.right
    
    #utility function to print the contents of the node
    def __str__(self):
        return self.data
    
    #utility function to check if a given node has 0 children (is a leaf node)
    def isLeaf(self):
        return ((self.left == None) and (self.right == None))

# Merkle tree class for actual implementation of the tree
class merkleTree:

    # function to initialise the tree with "None" as root and merkleRoot set to blank
    def __init__(self, arr):
        self.root = None
        self._merkleRoot = ''
        self.makeTreeFromArray(arr)
        self.calculateMerkleRoot()
        
    # private utility function to return a hash of an input string, using the sha256 algorithm    
    def __returnHash(self, x):
            return (hashlib.sha256(x.encode()).hexdigest())
        
    # function to generate a tree from a list. The list is expected to contain
    # the list of transactions(as strings) we wish to seal in a block    
    def makeTreeFromArray(self, arr):
        arr = arr.copy()
        # all the elements of the list must be leaf nodes, hence total nodes in the tree
        # is [2*(noOfLeafNodes) - 1]
        def __noOfNodesReqd(arr):
            x = len(arr)
            return (2*x - 1)
        
        # Private function to build tree with a list generated by a sequence corresponding to the
        # number of nodes required for the tree.
        # For e.g. if we have 4 transactions, i.e. 4 leafNodes, we will have a total of 7 nodes
        # in the tree. Hence this function takes in a list that looks like [1,2,3,4,5,6,7] and 
        # creates a binary tree by recursively inserting elements in inorder fashion.
        def __buildTree(arr, root, i, n): 
      
            # Base case for recursion  
            if i < n: 
                temp = Node(str(arr[i]))  
                root = temp  

                # insert left child  
                root.left = __buildTree(arr, root.left,2 * i + 1, n)  

                # insert right child  
                root.right = __buildTree(arr, root.right,2 * i + 2, n) 
                
            return root
        
        # This function adds our transactions (leaf Node data) to the Leaf Nodes in the tree
        # hence now the tree looks like :
                #                 ""
                #                 /\
                #         ""              ""
                #         /\              /\
                #    (txn1) (txn2)  (txn3) (txn4)      
        def __addLeafData(arr, node):
            
            if not node:
                return
            
            __addLeafData(arr, node.left)
            if node.isLeaf():
                node.data = self.__returnHash(arr.pop(0))
            else:
                node.data = ''
            __addLeafData(arr, node.right)
        
        #Driver function calls of the main function makeTreeFromArray
        nodesReqd = __noOfNodesReqd(arr)
        nodeArr = [num for num in range(1,nodesReqd+1)]
        self.root = __buildTree(nodeArr, None, 0, nodesReqd)
        __addLeafData(arr,self.root)

    # utility function to traverse the tree using inorder Traversal algorithm, must provide <the_name_of_the_tree_object>.root     
    def inorderTraversal(self, node):
        if not node:
            return []
        
        left = self.inorderTraversal(node.left)
        right = self.inorderTraversal(node.right)
        return left + [node.data] + right
    
    # function to calculate merkle root of the tree. This is a recursive algorithm.
    # Essentially, the data of parent node P, is the hash of concatenation of data of its two 
    # child nodes, A and B. If H(x) is the hash function,
    # P = H( H(A) + H(B) )
    # If R is the root, then data of R is the merkle hash or merkle root.  
    def calculateMerkleRoot(self):
         
        def __merkleHash(node):
            if node.isLeaf():
                return node
            
            left = __merkleHash(node.left).data
            right = __merkleHash(node.right).data
            node.data = self.__returnHash(left+right)
            return node
        
        merkleRoot = __merkleHash(self.root)
        self._merkleRoot = merkleRoot.data
        
        return self._merkleRoot 

    #utility function to get the merkle Root of that tree
    def getMerkleRoot(self):
        return self._merkleRoot
    
    # utility function to verify if the transactions are intact with respect to this tree.
    # Takes in a new list, creates a new merkle tree, calculate its merkle root,
    # and returns if merkle roots match. If they match, transaction list is intact and verified.
    # If transactions are tampered or changed, this function returns False.
    def verifyUtil(self, arr1):
        hash1 = self.getMerkleRoot()
        new_tree = merkleTree(arr1)
        hash2 = new_tree.getMerkleRoot()
        if hash1 == hash2:
            print(f"Hash 1: {hash1[:12]}, Hash 2: {hash2[:12]}") #hash shortened for readability
            print("Texts verified successfully")
            return True
        else:
            print(f"Hash 1: {hash1[:12]}, Hash 2: {hash2[:12]}") #hash shortened for readability
            print("Texts have been changed")
            return False
    
    def printTree(self):
        from collections import deque

        if not self.root:
            print("Empty tree.")
            return

        q = deque()
        q.append(self.root)
        level = 0
        while q:
            level_size = len(q)
            print(f"Level {level}: ", end='')
            for _ in range(level_size):
                node = q.popleft()
                print(f"{node.data[:10]} ", end='')  # Shorten the hash for readability
                if node.left:
                    q.append(node.left)
                if node.right:
                    q.append(node.right)
            print()
            level += 1


# make two lists from reading in files, change text in one of the files,
# create second list with same files but one with altered content
file_names = collect_files()
file_list1 = readinfiles(file_names)
change_text()
file_list2 = readinfiles(file_names)

#create the merkletrees
testtree1 = merkleTree(file_list1)
testtree2 = merkleTree(file_list2)

#Print merkle trees for original set of files and altered files
print("\n" +"Merkle tree of original files:")
testtree1.printTree()
print("\n" + "Merkle tree of same files with one altered text:")
testtree2.printTree()

#comparing the root hashes
print("\n" + "Comparing root hashes:")
#compare file_list1 against itself 
test1 = testtree1.verifyUtil(file_list1)
#compare file_list1 against file_list2
test2 = testtree1.verifyUtil(file_list2)


Please enter file name and path or type 'done':  project4part3question1testinput1.txt
Please enter file name and path or type 'done':  project4part3question1testinput2.txt
Please enter file name and path or type 'done':  project4part3question1testinput3.txt
Please enter file name and path or type 'done':  project4part3question1testinput4.txt
Please enter file name and path or type 'done':  project4part3question1testinput5.txt
Please enter file name and path or type 'done':  project4part3question1testinput6.txt
Please enter file name and path or type 'done':  done


Done with list


Please enter file name and path to change:  project4part3question1testinput6.txt
Please enter the new text:  Brand new test text!



Merkle tree of original files:
Level 0: 4a49ed2097 
Level 1: 323c6f387d 7e456d1de5 
Level 2: 2f297f1520 a2233f9fdf a140c0c1ed f31bd16c70 
Level 3: 1b4f0e9851 60303ae22b fd61a03af4 9c64347be9 

Merkle tree of same files with one altered text:
Level 0: 774047e8d7 
Level 1: 323c6f387d b0a941c743 
Level 2: 2f297f1520 a2233f9fdf a140c0c1ed 9bd47e2387 
Level 3: 1b4f0e9851 60303ae22b fd61a03af4 9c64347be9 

Comparing root hashes:
Hash 1: 4a49ed2097f0, Hash 2: 4a49ed2097f0
Texts verified successfully
Hash 1: 4a49ed2097f0, Hash 2: 774047e8d72d
Texts have been changed


## Question 2

After modifying the contents of one of the input files, the hash value in the corresponding leaf node will change, as will its parent node, and the parent node of that node, cascading up the tree including the root hash value.  However the other leaf node values will remain unchanged, and the parent nodes of any of two leaf nodes that do not include the hash for the changed file will also remain unchanged, except for the root hash value.  Since there are only 4 leaf values in this example, 3 leaf nodes will remain unchanged and the parent node that did include the hash of the changed file will also remain the same.  